Import des dépendences

In [1]:
import os
# if using Apple MPS, fall back to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image


Import du modèle

In [2]:
import torch
from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator

# checkpoint = "/home/killian/sam2/checkpoints/sam2.1_hiera_large.pt"
# checkpoint = "/home/killian/sam2/checkpoints/sam2.1_hiera_tiny.pt"
checkpoint = "/home/killian/sam2/checkpoints/sam2.1_hiera_small.pt"
# model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"
# model_cfg = "configs/sam2.1/sam2.1_hiera_t.yaml"
model_cfg = "configs/sam2.1/sam2.1_hiera_s.yaml"
# predictor = SAM2ImagePredictor(build_sam2(model_cfg, checkpoint))

sam2 = build_sam2(model_cfg, checkpoint, device="cuda", apply_postprocessing=False)

mask_generator = SAM2AutomaticMaskGenerator(sam2)
# with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    # predictor.set_image("/home/killian/sam2")
    # masks, _, _ = predictor.predict("0")
    # 🔹 Chemins vers le modèle et la configuration
#checkpoint = "./checkpoints/sam2_hiera_large.pt"
#model_cfg = "sam2_hiera_l.yaml"


permet de georéférencé nos images

In [3]:
import os
import tempfile

import kornia.augmentation as K
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
import torchvision.transforms as T

#from torchgeo.datasets import EuroSAT100
from torchgeo.datasets import RasterDataset, stack_samplesu modèle
from torchgeo.samplers import GridGeoSampler, RandomGeoSampler
from torchgeo.models import ResNet18_Weights, resnet18
from torchgeo.transforms import AugmentationSequential

import matplotlib.pyplot as plt
torch.manual_seed(0)

SyntaxError: invalid syntax (3855141078.py, line 11)

In [7]:
import sys
import os
import rasterio
from rasterio.transform import from_origin

def georeference_tiff(input_filename, output_filename):
    # Open the input raster file
    with rasterio.open(input_filename) as src:
        # Get raster dimensions
        width = src.width
        height = src.height

        # Set an arbitrary transformation
        transform = from_origin(0, 0, 1, 1)  # Adjust the pixel size as needed

        # Create the output raster profile with an arbitrary CRS (coordinate reference system)
        profile = src.profile
        profile.update(transform=transform, crs='EPSG:4326')  # EPSG:4326 is WGS 84

        # Modify output filename with "geo_" prefix
        output_filename = output_filename.replace(os.path.basename(output_filename), "geo_" + os.path.basename(output_filename))

        # Write to the output raster file
        with rasterio.open(output_filename, 'w', **profile) as dst:
            data = src.read()
            dst.write(data)

if __name__ == "__main__":
    if len(sys.argv) < 2:
        print("Usage: python script.py input_file1.tif input_file2.tif ...")
        sys.exit(1)

    input_files = sys.argv[1:]
    for input_file in input_files:
        output_file = "geo_" + os.path.basename(input_file)
        
        georeference_tiff('/home/killian/data2025/15485/X200_15485_PB1.tif', '/home/killian/data2025/15485/X200_15485_PB1.tif')
        print(f"Georeferenced TIFF saved as {output_file}")
 

/home/killian/.local/lib/python3.10/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/home/killian/.local/lib/python3.10/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: The given matrix is equal to Affine.identity or its flipped counterpart. GDAL may ignore this matrix and save no geotransform without raising an error. This behavior is somewhat driver-specific.
  dataset = writer(


Georeferenced TIFF saved as geo_kernel-v3f11e16ea8e2f7ea713cb7064461c49ae2c3a1199.json


on définit le DataLoader

In [3]:
import os
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as T
# import rasterio
import matplotlib.pyplot as plt
import tifffile

class CustomDataset(Dataset):

    def __init__(self,input_raster, tile_size=640, stride=10,transforms=None):

        # raster = rasterio.open(input_raster).read()
        # raster = plt.imread(input_raster)
        raster = tifffile.imread(input_raster)
        
        self.raster = raster[:,:tile_size,:]
        print(self.raster.shape)
        self.tile_size = tile_size
        self.stride = stride
        self.transforms = transforms

    def __len__(self):
        return int(self.raster.shape[0]/self.tile_size)
        

    def __getitem__(self, idx):
        
        if idx == 0:
            start_y = 0
            end_y = self.tile_size
        
        else:
            start_y = idx*self.tile_size - self.stride
            end_y = (idx+1)*self.tile_size - self.stride

        data = self.raster[start_y:end_y,:,:]
        data = data.swapaxes(0,2)
        # data = (data - np.min(data)) / (np.max(data) - np.min(data))
        # if self.transforms:
        #     data = self.transforms(data)
        return data
        
 

Dataloader et dataset

In [4]:
from torch.utils.data import DataLoader

# dataset = CustomDataset('/home/killian/data2025/15485/X200_15485_PB1.tif', tile_size=640, stride=10)
dataset = CustomDataset('/home/killian/data2025/TGV4/X200_TGV4A_B-P.tif', tile_size=640, stride=10)
dataloader = DataLoader(dataset,batch_size=1, shuffle=False)


(8935, 640, 3)


Hyperparametres modèles

In [ ]:
#Hyperparametres
model = SAM2AutomaticMaskGenerator(
    model=sam2,
    points_per_side=50,  # Plus de points pour capturer les details
    points_per_batch=50,  # Augmenter pour calculer le nbre de points pris en meme temps (/!\ GPU)
    pred_iou_thresh=0.5,  # Reduire pour accepter plus de mask
    stability_score_thresh=0.70,  # Rzduire pour ne pas exclure trop de mask
    stability_score_offset=0.8,
    crop_n_layers=6,  # Ammeliore la segmentation des petites structures
    box_nms_thresh=0.70,  # Eviter la suppression excessive de petite structure
    crop_n_points_downscale_factor=1.2,  # Adapter aux images a haute resolution
    min_mask_region_area=20.0,  # Conserver plus de petits objets
    use_m2m=True,  # Mode avancé 
)

In [11]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import csv
import cv2

device = "cuda" if torch.cuda.is_available() else "cpu"

# Dossier de sortie
output_dir = "/home/killian/sam2/inferences"
os.makedirs(output_dir, exist_ok=True)

# Fichiers CSV pour sauvegarder les masks
csv_masks_file = os.path.join(output_dir, "mask_measurements.csv")

with open(csv_masks_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Image_ID", "Mask_ID", "Centroid_X", "Centroid_Y", "Area", "Equivalent_Diameter"])

size_threshold = 560  # Seuil pour filtrer les objets trop grands(0.5060 px/µm) donc <280µm

# Boucle d'inférence
for i, image in enumerate(dataloader):
    print(f"Image {i+1}/{len(dataloader)} - Avancement : {i/len(dataloader):.2%}", end='\r')
    
    image_np = image.squeeze(0).cpu().numpy()
    image_np = image_np.swapaxes(0, 2)
    pred = model.generate(image_np)
    
    res_tensor = torch.stack([torch.tensor(m['segmentation'], dtype=torch.bool) for m in pred])
    filtered_tensor = res_tensor[res_tensor.sum(dim=(1, 2)) <= size_threshold]
    res_merge = filtered_tensor.any(dim=0).to(device)
    
    # Trier les masques de droite à gauche selon la coordonnée X de leur centroïde
    masks_info = []
    for mask_id, mask_pred in enumerate(filtered_tensor):
        mask_np = mask_pred.cpu().numpy().astype(np.uint8)
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask_np, connectivity=8)
        
        for j in range(1, num_labels):  # On ignore le label 0 (arrière-plan)
            x, y = centroids[j]
            area = stats[j, cv2.CC_STAT_AREA]
            equivalent_diameter = np.sqrt(4 * area / np.pi)
            masks_info.append((x, y, area, equivalent_diameter, mask_id))
    700
    # Trier les masques en fonction de X (croissant)
    masks_info.sort(reverse=True, key=lambda m: m[0])
    
    # Sauvegarde des masques triés
    with open(csv_masks_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        for x, y, area, equivalent_diameter, mask_id in masks_info:
            writer.writerow([f"Image_{i}", mask_id, x, y, area, equivalent_diameter])
    
    # Sauvegarde de l'image de prédiction et de l'image originale
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(image_np)
    plt.title("Image Originale")
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(res_merge.cpu().numpy(), cmap='gray')
    plt.title("Masques Prédits")
    plt.axis('off')
    
    plt.savefig(os.path.join(output_dir, f"Image_{i}.png"))
    plt.close()
